# Hey, maikol here

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### File functions

`make_dirs`

In [23]:
from maikol_utils.file_utils import make_dirs

dirs = ["./data/input", "./data/output"]
# Safely creates all these directories if they don't exist
make_dirs(dirs)

`check_dirs_existance`

In [24]:
from maikol_utils.file_utils import check_dirs_existance

dirs = ["./data", "./config", "./logs"]
# Raises KeyError if any path is missing
try:
    check_dirs_existance(dirs)
except KeyError as e:
    print(e)  # e.g. "Some paths were not found: ['./config', './logs']"


"Some paths were not found: ['./config', './logs']"


`save_json` & `load_json`

In [25]:
from maikol_utils.file_utils import save_json, load_json

data = {"users": ["alice", "bob"], "count": 2}
# Saves to outputs/users.json, creating the folder if needed
save_json("outputs/users.json", data)

# Attempt to load; if not found, returns {}
settings = load_json("outputs/users.json")
print(settings)  # {"users": ["alice", "bob"], "count": 2}

Saving output at outputs/users.json...
Loading output from outputs/users.json...
{'users': ['alice', 'bob'], 'count': 2}


### Printing

`print_separator`

In [26]:
from maikol_utils.print_utils import print_separator

print_separator("START PROCESS", sep_type="START")
print_separator("STEP COMPLETE", sep_type="SHORT")
print_separator("FINAL RESULTS", sep_type="SUPER")



                                                         START PROCESS                                                          


________________________________
         STEP COMPLETE          

                                                         FINAL RESULTS                                                          



`print_color` & `print_warn`

In [27]:
from maikol_utils.print_utils import print_color, print_warn, print_error

print_color("All systems go", color="green")
print_color("Unexpected value", color="red")

# Prints ⚠️Missing config file!⚠️ in yellow
print_warn("Missing config file!")

# Prints ❌Missing config file!❌ in yellow
text = print_error("No model found!")

All systems go
Unexpected value
⚠️Missing config file!⚠️
❌No model found!❌


`print_status`
- this wont work here :)

In [ ]:
import time
from maikol_utils.print_utils import print_status, clear_status

print("Processing document...")


n_files = 3
for i in range(1, n_files+1):
    print_status(f" - Processing page {i}/{n_files}...")
    time.sleep(0.5)
# At the end call clear_status() to clear the previous line
clear_status()

print("DONE!")


"""
>>> print("Processing document...")
>>> Processing document...

>>> print_status(f" - Processing page {i}/{n_files}...")
>>> Processing document...
>>>  - Processing page 1/3

>>> print_status(f" - Processing page {i}/{n_files}...")
>>> Processing document...
>>>  - Processing page 2/3

>>> print_status(f" - Processing page {i}/{n_files}...")
>>> Processing document...
>>>  - Processing page 3/3

>>> clear_status()
>>> print("DONE!")
>>> Processing document...
>>> DONE!
"""

`clear_bash` & `print_clear_bash`
- this wont work here :)

In [ ]:
import time
from maikol_utils.print_utils import clear_bash, print_clear_bash


print("Line A")
time.sleep(1)
print("Line B")
time.sleep(1)
clear_bash(2)   # removes both lines from the terminal
print("Clean slate!")


print("Loading data...")
time.sleep(1)
print_clear_bash("✔️ Data loaded successfully!", n_lines=1)

"""

>>> print("Line A")
>>> Line A

>>> print("Line B")
>>> Line A
>>> Line B

>>> clear_bash(2) 
>>> 

>>> print("Clean slate!")
>>> Clean state!

>>> print("Loading data...")
>>> Clean state!
>>> Loading data...

>>> print_clear_bash("✔️ Data loaded successfully!", n_lines=1)
>>> Clean state!
>>> ✔️ Data loaded successfully!
"""


# With logger

### Configuration

In [30]:
import os
import time
import logging
from dataclasses import dataclass

from maikol_utils.print_utils import clear_bash, print_clear_bash, set_logger, print_log
from logging.handlers import TimedRotatingFileHandler


# ======================================================================================================               
#                                              CONFIGURATION
# ======================================================================================================
@dataclass
class Configuration:
    logs_folder:           str = r"logs"
    logs_path_file: str = fr"{logs_folder}/app.log"

# ======================================================================================================               
#                                              LOGGER FOR PRINT
# ======================================================================================================
class ColorFormatter(logging.Formatter):
    COLORS = {
        logging.DEBUG:    "\033[32m",  # green
        logging.INFO:     "\033[34m",  # blue
        logging.WARNING:  "\033[33m",  # yellow-orange
        logging.ERROR:    "\033[31m",  # red
        logging.CRITICAL: "\033[1;31m" # bright red
    }
    RESET = "\033[0m"

    def format(self, record):
        color = self.COLORS.get(record.levelno, "")
        record.levelname = f"{color}{record.levelname}{self.RESET}"
        return super().format(record)

os.makedirs(Configuration.logs_folder, exist_ok=True)
file_handler = TimedRotatingFileHandler(
    Configuration.logs_path_file, when="midnight", interval=1, backupCount=7, encoding="utf-8"
)
file_handler.setFormatter(logging.Formatter(
    "%(asctime)s | %(levelname)s: %(message)s", "%Y-%m-%d %H:%M"
))

# Create shared handler with your formatter
handler = logging.StreamHandler()
handler.setFormatter(ColorFormatter(
    "%(asctime)s | %(levelname)s: %(message)s", "%Y-%m-%d %H:%M"
))

# Apply to your logger
logger = logging.getLogger(__name__)
logger.handlers.clear()
logger.propagate = False
logger.setLevel(logging.INFO)
logger.addHandler(handler)
logger.addHandler(file_handler)

# Also apply to FastAPI and Uvicorn loggers
for name in ["uvicorn", "uvicorn.access", "uvicorn.error"]:
    log = logging.getLogger(name)
    log.handlers.clear()
    log.propagate = False
    log.setLevel(logging.INFO)
    log.addHandler(handler)

### Example

In [ ]:

set_logger(logger)  
print_log("Line A")
time.sleep(1)
print_log("Line B")
time.sleep(1)
clear_bash(2)   # removes both lines from the terminal
print_log("Clean slate!")


print_log("Loading data...")
time.sleep(1)
print_clear_bash("✔️ Data loaded successfully!", n_lines=1)

"""

>>> print("Line A")
>>> 2025-07-21 09:53 | INFO: Line A

>>> print("Line B")
>>> 2025-07-21 09:53 | INFO: Line A
>>> 2025-07-21 09:53 | INFO: Line B

>>> clear_bash(2) 
>>> 

>>> print("Clean slate!")
>>> 2025-07-21 09:53 | INFO: Clean state!

>>> print("Loading data...")
>>> 2025-07-21 09:53 | INFO: Clean state!
>>> 2025-07-21 09:53 | INFO: Loading data...

>>> print_clear_bash("✔️ Data loaded successfully!", n_lines=1)
>>> 2025-07-21 09:53 | INFO: Clean state!
>>> 2025-07-21 09:53 | INFO: ✔️ Data loaded successfully!
"""
